In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

def get_model():
    # Create a simple model.
    inputs = keras.Input(shape=(32,))
    outputs = keras.layers.Dense(1)(inputs)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="adam", loss="mean_squared_error")
    return model


model = get_model()

# Train the model.
test_input = np.random.random((128, 32))
test_target = np.random.random((128, 1))
model.fit(test_input, test_target)

# Calling `save('my_model')` creates a SavedModel folder `my_model`.
model.save("my_model/1")

4/4 [==============================] - 0s 2ms/step - loss: 0.3722
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: my_model/1/assets


In [2]:
# model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________


In [3]:
!saved_model_cli show --dir my_model/1 --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 32)
        name: serving_default_input_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Defined Functions:
  Function Name: '__call__'
    Option #

In [9]:
# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0   2246      0  0:00:01  0:00:01 --:--:--  2246
OK
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3012 B]
Get:2 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server-universal amd64 Packages [349 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease                        
Get:5 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 Packages [341 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:7 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [

In [10]:
!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tensorflow-model-server
0 upgraded, 1 newly installed, 0 to remove and 64 not upgraded.
Need to get 187 MB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable/tensorflow-model-server amd64 tensorflow-model-server all 2.2.0 [187 MB]
Fetched 187 MB in 16s (11.6 MB/s)                                              
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package tensorflow-model-server.
(Reading database ... 20449 files and directories currently installed.)
Preparing to unpack .../tensorflow-model-server_2.2.0_all.deb ...
Unpacking tensorflow-model-server (2.2.0) ...
Setting up tensorflow-model-server (2.2.0) ...


In [ ]:
# tensorflow_model_server   --rest_api_port=8501   --model_base_path="/tf/develop/tensorflow-101/experts/code_sample/chapter-7/saved_model"tensorflow_model_server   --rest_api_port=8501   --model_name=test   --model_base_path="/tf/develop/tensorflow-101/experts/code_sample/chapter-7/my_model"

In [4]:
import json

test_input = np.random.random((128, 32)).tolist()

data = json.dumps({"signature_name": "serving_default", "instances": test_input})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

Data: {"signature_name": "serving_default", "instances": ... 8093713, 0.04353883385346169, 0.44587759367421853]]}


In [6]:
import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/test:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']
print(predictions)

[[0.305242449], [0.0115488935], [0.0236730333], [-0.255960315], [0.216458246], [-0.388122469], [0.250144303], [0.148375377], [0.135872915], [0.410658926], [0.342577934], [0.501954496], [0.58278662], [0.427872568], [-0.132328615], [0.711528778], [0.0487683564], [1.13452196], [0.478393227], [-0.438883632], [0.211962774], [0.003521787], [0.32265994], [-0.0941061825], [-0.217286453], [0.104702607], [-0.222778484], [0.364578873], [-0.371712893], [-0.137581781], [0.36497578], [0.686913073], [-0.216299668], [0.0538681298], [-0.185493156], [-0.0428354591], [-0.169320807], [0.347735375], [-0.1134886], [-0.330542773], [-0.0950512439], [0.533949912], [0.756000519], [0.167955056], [0.121560022], [0.534835517], [0.427884489], [0.278388292], [-0.282165617], [-0.109700814], [0.878007054], [-0.00586619787], [0.441364497], [0.180021688], [0.194043145], [0.556959808], [0.494386613], [0.0912641734], [0.404759824], [0.303757399], [0.00963546522], [0.395944834], [-0.00162012689], [-0.32002452], [-0.2868450